In [ ]:


import json
import math

import pandas as pd
import numpy as np
import folium

In [ ]:
wales = pd.read_excel('data/wales-approved-projects-2015.xlsx')
wales.head()

In [ ]:
wales = wales[~wales['EU Programme'].isnull()]

In [ ]:
grid = pd.read_csv('data/grid20180625/grid.csv')
grid.head()

In [ ]:
grid_addresses = pd.read_csv('data/grid20180625/full_tables/addresses.csv', low_memory=False)
grid_addresses.head()

In [ ]:
wales = pd.merge(wales, grid, how='left', left_on='Lead Organisation', right_on='Name')
wales.shape

In [ ]:
wales_edu = pd.merge(wales, grid_addresses, how='left', left_on='ID', right_on='grid_id')
wales.shape

In [ ]:
wales_edu[wales.ID.isnull()].shape

In [ ]:
wales[wales.ID.isnull()][['Lead Organisation']].sort_values(axis=0, by='Lead Organisation').tail(40)

In [ ]:
walesLocations = pd.DataFrame([('Aberystwyth Innovation and Enterprise Campus Ltd', 'SY233EE'),
           ('Blaenau Gwent County Borough Counci', 'NP236XB'),
           ('Bridgend County Borough Council', 'CF314WB'),
           ('Caerphilly County Borough Council', 'CF827PG', 'CF827PG', 'CF827WF'),
           ('Carmarthenshire County Council', 'SA183EL'),
           ('Ceredigion County Council', 'SY233UE'),
           ('Chwarae Teg', 'CF245JW'),
           ('Coleg Gwent', 'NP19 4TS'),
           ('Coleg Y Cymoedd', 'CF448EN', 'CF818RD', 'CF157QY', 'CF827XR'),
           ('Conwy County Borough Council', 'LL328DU'),
           ('Denbighshire County Borough Council', 'LL159AZ'),
           ('Engineering Education Scheme Wales', 'CF313WT'),       
           ('Finance Wales Plc', 'CF104BZ', 'SA148LQ', 'SY161RB', 'LL170JD'),  
            ('Grwp Llandrillo-Menai', 'LL284HZ', 'LL572TP', 'LL402SW'),
           ('Isle of Anglesey County Council', 'LL777TW'),
            ('Marine Power Systems Limited', 'SA18AS'),
            ('Menter Mon', 'LL777LR'),
            ('Minesto UK LTD', 'LL651UN', 'BT221NZ'),
            ('Neath Port Talbot County Borough Council', 'SA131PJ'), 
            ('Newport City Council', 'NP204UR'),
            ('Pembrokeshire Coastal Forum', 'SA726UL'),
            ('Pembrokeshire County Council', 'SA611TP'), 
            ('Powys County Council', 'LD15LG'),
            ('Rhondda Cynon Taff Council', 'CF402XX'),
            ('SOVA', 'CF101FE'),
            ('Swansea Council', 'SA13SN'),
            ('TWI Ltd', 'SA131SB'),
            ('Torfaen County Borough Council', 'NP46YB'),
            ('University of Wales Trinity Saint David', 'SA487ED'),
            ('WG - Department for Education and Skills', 'CF103NQ'),
            ('WG - Department for Health and Social Services', 'CF103NQ'),
            ('WG - Department of Local Government and Communities', 'CF103NQ'),
    ('WG - Department for Economy, Science and Transport', 'CF103NQ'),      
           ('Wales Co operative Centre', 'CF832AX'),
            ('Wales Council for Voluntary Action', 'CF832AX', 'CF105FH', 'SY233AH'),
            ('Wave Hub Limited', 'TR274DD'),
            ('Wave-Tricity Limited', 'SA726YH'),
            ('Welsh Contact Centre Forum Ltd', 'CF642AG')

 
], 
          
          columns=['Lead Organisation', 'postcode1', 'postcode2', 'postcode3', 'postcode4'])
walesLocations.head()

Not found             ('WAG The Economic Development Group within DEandT'), WAG DEandT WEFO

In [ ]:
wales_nonedu = pd.merge(wales, walesLocations, how='left', left_on='Lead Organisation', right_on='Lead Organisation')
wales_nonedu.shape

In [ ]:
postcodes = pd.read_csv('data/ukpostcodes.csv')

In [ ]:
postcodes.head()

In [ ]:
postcodes['postcode'] = postcodes.postcode.str.strip().str.replace(' ', '')

In [ ]:
wales_nonedu =  pd.merge(wales_nonedu, postcodes,  left_on=['postcode1'], right_on=['postcode'])

In [ ]:
wales_nonedu.columns

In [ ]:
wales_edu.columns

In [ ]:
wales.columns

In [ ]:
def make_wales_data_geo_json(coordis_data):
    def make_feature(row):
        properties = {
            property: row[property]
            for property in ['Project Title','Lead Organisation', 'Project Description',  'EU funds awarded', 'Total Project cost']
            if str(row[property]) != 'nan'
        }
        properties['Project start date'] = str(row['Project start date'])
        properties['Project end date'] = str(row['Project end date'])
        return {
            'type': 'Feature',
            'geometry': {
                "type": "Point",
                "coordinates": [row['longitude'], row['latitude']]
            },
            'properties': properties
        }
    features = list(coordis_data.apply(make_feature, axis=1))
    return { 'type': 'FeatureCollection', 'features': features }
with open('data/wales_data.geo.json', 'w') as file:
    json.dump(make_wales_data_geo_json(
        wales_nonedu[~pd.isnull(wales_nonedu['latitude'])]
    ), file, sort_keys=True)

In [ ]:
def make_wales_edu_data_geo_json(coordis_data):
    def make_feature(row):
        properties = {
            property: row[property]
            for property in ['Project Title','Lead Organisation', 'Project Description',  'EU funds awarded', 'Total Project cost']
            if str(row[property]) != 'nan'
        }
        properties['Project start date'] = str(row['Project start date'])
        properties['Project end date'] = str(row['Project end date'])
        return {
            'type': 'Feature',
            'geometry': {
                "type": "Point",
                "coordinates": [row['lng'], row['lat']]
            },
            'properties': properties
        }
    features = list(coordis_data.apply(make_feature, axis=1))
    return { 'type': 'FeatureCollection', 'features': features }
with open('data/wales_edu_data.geo.json', 'w') as file:
    json.dump(make_wales_edu_data_geo_json(
        wales_edu[~pd.isnull(wales_edu['lat'])]
    ), file, sort_keys=True)